In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import json

# Read the JavaScript data from the file
with open('main.js', 'r') as file:
    js_code = file.read()


# Extract the JavaScript array parts
start_index1 = js_code.find('[')
end_index1 = js_code.find(']', start_index1)
js_array_str1 = js_code[start_index1:end_index1 + 1]

start_index2 = js_code.find('[', end_index1)
end_index2 = js_code.find(']', start_index2)
js_array_str2 = js_code[start_index2:end_index2 + 1]

# Convert the JavaScript array strings to Python lists
numbers1 = json.loads(js_array_str1)
numbers2 = json.loads(js_array_str2)

print("Red Team:", numbers1)  # Output: [1, 2, 3, 4, 5]
print("Blue Team:", numbers2)  # Output: [6, 7, 8, 9, 10]


Red Team: [200, 600, 500, 600, 600]
Blue Team: [7000, 600, 600, 600, 400]


In [10]:
team1_features = np.array([numbers1])  # Example feature vector for Team 1
team2_features = np.array([numbers2])  # Example feature vector for Team 2
labels = np.array([1])  # Example label (1 for Team 1 win)

# Split data into training and testing sets
X_train = np.concatenate((team1_features, team2_features))
y_train = np.repeat(labels, 2)  # Duplicate labels for both teams
X_test = np.concatenate((team1_features, team2_features))

# Train linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict probabilities for the test set
probabilities = model.predict(X_test)